# Hw-agent: Nutrition Assistant

LangChain + Qwen2.5-3B-Instruct + Tool Calling

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
import os

API_NINJAS_KEY = os.environ['API_NINJAS_KEY']
CALORIE_NINJAS_KEY = os.environ['CALORIE_NINJAS_KEY']

In [19]:
import requests
import re
from langchain_core.tools import tool

def parse_amount(text):
    """Parse ingredient amount including fractions like '3/4 c', '1 1/2 cups'."""
    text = text.strip().lower()
    
    fraction_map = {
        '1/4': 0.25, '1/2': 0.5, '3/4': 0.75,
        '1/3': 0.33, '2/3': 0.67,
        '1/8': 0.125, '3/8': 0.375, '5/8': 0.625, '7/8': 0.875
    }
    
    # First check if starts with fraction only (no whole number before it)
    frac_only_match = re.match(r'^(\d+/\d+)\s*(c|cup|cups|tb|tbsp|ts|tsp|lb|lbs|oz)?\b', text)
    if frac_only_match:
        frac_str = frac_only_match.group(1)
        return fraction_map.get(frac_str, 0)
    
    # Then check: whole number + optional fraction + unit
    mixed_match = re.match(r'^(\d+)\s+(\d+/\d+)\s*(c|cup|cups|tb|tbsp|ts|tsp|lb|lbs|oz)?\b', text)
    if mixed_match:
        whole = int(mixed_match.group(1))
        frac = fraction_map.get(mixed_match.group(2), 0)
        return whole + frac
    
    # Just a whole number + unit
    whole_match = re.match(r'^(\d+)\s*(c|cup|cups|tb|tbsp|ts|tsp|lb|lbs|oz)?\b', text)
    if whole_match:
        return int(whole_match.group(1))
    
    # Just a decimal number
    num_match = re.match(r'^([\d.]+)', text)
    if num_match:
        return float(num_match.group(1))
    
    return None

@tool
def get_recipe(dish_name: str) -> str:
    """Get recipe for a dish: ingredients list and servings count."""
    url = f"https://api.api-ninjas.com/v1/recipe?query={dish_name}"
    headers = {'X-Api-Key': API_NINJAS_KEY}
    response = requests.get(url, headers=headers, timeout=10)
    recipes = response.json()
    if recipes:
        r = recipes[0]
        ingredients_raw = r.get('ingredients', '')
        ingredients = ingredients_raw.split('|')
        ingredients_lower = ingredients_raw.lower()
        
        # Parse amounts for each ingredient
        parsed = []
        for ing in ingredients:
            ing = ing.strip()
            amount = parse_amount(ing)
            if amount is not None:
                parsed.append(f"  - {ing} [amount={amount}]")
            else:
                parsed.append(f"  - {ing}")
        
        # Check common ingredients - explicit 1/0
        checks = []
        check_items = ['egg', 'oil', 'olive oil', 'butter', 'salt', 'cheese', 'parmesan', 
                       'onion', 'carrot', 'water', 'wine', 'lemon', 'lime', 'soda', 
                       'raisin', 'tomato', 'flour', 'baking soda']
        for item in check_items:
            found = item in ingredients_lower
            checks.append(f"HAS_{item.upper().replace(' ','_')}: {'1' if found else '0'}")
        
        result = f"Title: {r.get('title')}\n"
        result += f"Servings: {r.get('servings', '1')}\n"
        result += f"Ingredients_count: {len(ingredients)}\n"
        result += "Ingredients:\n" + "\n".join(parsed) + "\n"
        result += "Checks: " + ", ".join(checks)
        return result
    return f"Recipe for '{dish_name}' not found"

@tool  
def get_nutrition(food_query: str) -> str:
    """Get calories and nutrients for foods. Example: '100g caesar salad'."""
    url = f"https://api.calorieninjas.com/v1/nutrition?query={food_query}"
    headers = {'X-Api-Key': CALORIE_NINJAS_KEY}
    response = requests.get(url, headers=headers, timeout=10)
    data = response.json()
    if data.get('items'):
        total = {'calories': 0, 'protein_g': 0, 'fat_g': 0, 'carbs_g': 0, 'sugar_g': 0, 'cholesterol_mg': 0}
        for item in data['items']:
            total['calories'] += item.get('calories', 0)
            total['protein_g'] += item.get('protein_g', 0)
            total['fat_g'] += item.get('fat_total_g', 0)
            total['carbs_g'] += item.get('carbohydrates_total_g', 0)
            total['sugar_g'] += item.get('sugar_g', 0)
            total['cholesterol_mg'] += item.get('cholesterol_mg', 0)
        return f"Calories: {total['calories']:.2f} kcal, Protein: {total['protein_g']:.2f}g, Fat: {total['fat_g']:.2f}g, Carbs: {total['carbs_g']:.2f}g, Sugar: {total['sugar_g']:.2f}g, Cholesterol: {total['cholesterol_mg']:.2f}mg"
    return f"Nutrition info for '{food_query}' not found"

tools = [get_recipe, get_nutrition]

In [20]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(model="qwen2.5:3b", temperature=0)

In [21]:
from langgraph.prebuilt import create_react_agent

system_prompt = """You are a nutrition assistant. Always use tools to get accurate data.

QUESTION TYPES:

1. CALORIES/NUTRIENTS: Use get_nutrition("Xg dish_name")
   - Return the exact number from the response
   
2. INGREDIENT COUNT: Use get_recipe(dish_name)
   - Return Ingredients_count value
   
3. INGREDIENT AMOUNT: Use get_recipe(dish_name)
   - Find the ingredient line with [amount=X]
   - Return X (the parsed amount)
   - "3/4 c olive oil [amount=0.75]" -> answer: 0.75
   - "1 c flour [amount=1]" -> answer: 1
   
4. YES/NO INGREDIENT QUESTIONS: Use get_recipe(dish_name)
   - Look at "Checks:" line
   - HAS_EGG: 1 means egg is present -> answer: 1
   - HAS_EGG: 0 means no egg -> answer: 0
   - For "contains egg?" -> find HAS_EGG value
   - For "contains oil?" -> find HAS_OIL value
   - For "contains butter?" -> find HAS_BUTTER value

5. COMPARISON: Use get_nutrition for both dishes (100g each)
   - Return 1 if first has more, 2 if second has more

OUTPUT: Only the final number, rounded to 2 decimals."""

agent = create_react_agent(chat_model, tools, prompt=system_prompt)

/tmp/ipykernel_36091/3508945438.py:32: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(chat_model, tools, prompt=system_prompt)


In [22]:
# Test problematic cases
test_questions = [
    "Does Caesar salad contain egg?",  # Should be 1
    "How many cups of flour will I need to cook blueberry pancakes?",  # Should be 1
    "How many cups of olive oil do I need to cook Caesar salad?",  # Should be 0.75
]

for q in test_questions:
    print(f"\nQ: {q}")
    result = agent.invoke({"messages": [("human", q)]})
    for msg in result["messages"]:
        if msg.type == "tool":
            print(f"TOOL: {msg.content[:500]}...")
        elif msg.type == "ai" and msg.content:
            print(f"ANSWER: {msg.content}")


Q: Does Caesar salad contain egg?
TOOL: Title: Vincent Price's Caesar Salad for a Busy Cook
Servings: 4 Servings
Ingredients_count: 13
Ingredients:
  - 2 Heads romaine lettuce; washed and dried [amount=2]
  - 1 Clove garlic;, peeled [amount=1]
  - 3/4 c Olive oil [amount=0.75]
  - 1 ds Cayenne pepper [amount=1]
  - 1 ds Tabasco® sauce [amount=1]
  - 1 ts Sugar [amount=1]
  - 1 10" strip anchovy paste [amount=1]
  - Freshly ground black pepper
  - 1/2 ts Salt [amount=0.5]
  - 1 Whole egg [amount=1]
  - 1 lg Lemon;, juiced [amount=1]
  ...
ANSWER: 1

Q: How many cups of flour will I need to cook blueberry pancakes?
TOOL: Title: Peach/blueberry Pancakes
Servings: 1 Servings
Ingredients_count: 8
Ingredients:
  - 1 c Flour (whole wheat is best) [amount=1]
  - 1 c Soy milk (low fat) [amount=1]
  - 2 ts Baking powder [amount=2]
  - 1/2 ts Salt [amount=0.5]
  - 1 ts Cinnamon [amount=1]
  - 1 tb Maple syrup [amount=1]
  - 2 sm Ripe peaches -or- (up to 3) [amount=2]
  - 1 lg Ripe peaches (up to

In [23]:
import pandas as pd
import re

test_df = pd.read_csv('test.csv')

def extract_number(text, question=""):
    text = str(text).strip()
    question_lower = question.lower()
    
    # Common fraction map
    fraction_map = {
        '1/4': 0.25, '1/2': 0.5, '3/4': 0.75,
        '1/3': 0.33, '2/3': 0.67,
        '1/8': 0.125, '3/8': 0.375, '5/8': 0.625, '7/8': 0.875
    }
    
    # Check for fractions in text
    for frac, val in fraction_map.items():
        if frac in text:
            return round(val, 2)
    
    # Extract all numbers from text
    numbers = re.findall(r'[-+]?\d*\.?\d+', text)
    if not numbers:
        return 0.0
    
    # For yes/no questions (contain, include, has, is there), return first valid answer (0 or 1)
    is_yes_no = any(kw in question_lower for kw in ['contain', 'include', 'is there', 'does', 'do '])
    if is_yes_no and not any(kw in question_lower for kw in ['how many', 'how much', 'what is']):
        for num in numbers:
            val = float(num)
            if val in [0, 1]:
                return val
        # If text says yes/contains, return 1; if no/not, return 0
        if any(w in text.lower() for w in ['yes', 'contains', 'includes', 'present']):
            return 1.0
        if any(w in text.lower() for w in ['no', 'not', 'does not']):
            return 0.0
        return float(numbers[-1]) if numbers else 0.0
    
    # For comparison questions, return 1 or 2
    is_comparison = any(kw in question_lower for kw in ['which dish has more', 'which has more'])
    if is_comparison:
        for num in numbers:
            val = float(num)
            if val in [1, 2]:
                return val
        # Default to last number
        return round(float(numbers[-1]), 2)
    
    # For all other questions, return the last number
    return round(float(numbers[-1]), 2)

def ask_agent(question):
    try:
        result = agent.invoke({"messages": [("human", question)]})
        answer = result["messages"][-1].content
        return extract_number(answer, question)
    except Exception as e:
        print(f"Error: {e}")
        return 0.0

results = []
for idx, row in test_df.iterrows():
    print(f"{idx+1}/{len(test_df)}: {row['question'][:50]}...")
    answer = ask_agent(row['question'])
    results.append({'id': row['id'], 'y_true': answer})
    print(f"  -> {answer:.2f}")

submission = pd.DataFrame(results)
submission.to_csv('submission.csv', index=False, float_format='%.2f')
print("Saved to submission.csv")

1/100: How many calories are there in 100 gramm of Caesar...
  -> 160.40
2/100: How many ingredients are in lasagna?...
  -> 15.00
3/100: How many apples do I need to cook apple pie?...
  -> 4.00
4/100: How many calories are in 100 gramm of apple pie?...
  -> 240.10
5/100: How much fat is in 100 gramm of grilled chicken?...
  -> 3.50
6/100: How much protein is in 450 gramm of Mushroom soup?...
  -> 6.10
7/100: Does grilled chicken contain oil?...
  -> 1.00
8/100: How many ingredients are in apple pie?...
  -> 9.00
9/100: How many calories are in 100 gramm of grilled chic...
  -> 152.40
10/100: How much protein is in 100 gramm of Caesar salad?...
  -> 3.40
11/100: How many shallots are required to cook grilled chi...
  -> 2.00
12/100: Does Mushroom soup include butter?...
  -> 1.00
13/100: How much fat is in 100 gramm of apple pie?...
  -> 11.00
14/100: What is the carbohydrate content in 100 gramm of b...
  -> 29.40
15/100: How much protein is in 260 gramm of Caesar salad?...
  -> 8.70